# Explore what Pelicanfs is

- An extension on `fsspec`
- `fsspec` can treat `s3` as a "local" file system
- Common methods: .open(), .ls(), .exists(), .rm(), .cat(), .put(), .get(), etc.

## Notes

- Maybe I can bypass Pelican layer (s3 > Pelican > fsspec > loader) and directly develop with s3 (s3 > fsspec > loader), the final integration should be a drop-in replacement with `pelicanfs`
- Need to understand better about the async and caching layer I guess? But I assume `pelicanfs` will handle that gracefully? 
- Many broken examples


### Not important

- Good to know: Can `fsspec` works with `pathlib` interface?


In [40]:
from pelicanfs.core import PelicanFileSystem

In [ ]:
file_system = PelicanFileSystem("pelican://osg-htc.org")
test_dir = "/ospool/uc-shared/public/OSG-Staff/validation"


b'Hello, World!\n'


In [43]:
file_system.ls(test_dir)

[{'name': '/ospool/uc-shared/public/OSG-Staff/validation/test.txt',
  'size': None,
  'type': 'file'},
 {'name': '/ospool/uc-shared/public/OSG-Staff/validation/test2.txt',
  'size': None,
  'type': 'file'},
 {'name': '/ospool/uc-shared/public/OSG-Staff/validation/README.txt',
  'size': None,
  'type': 'file'}]

In [47]:
file_system.find(test_dir)

['/ospool/uc-shared/public/OSG-Staff/validation/README.txt',
 '/ospool/uc-shared/public/OSG-Staff/validation/test.txt',
 '/ospool/uc-shared/public/OSG-Staff/validation/test2.txt']

In [52]:
file_system.find(test_dir, max_depth=2)

TypeError: PelicanFileSystem._ls_real() got an unexpected keyword argument 'max_depth'

In [49]:
file_system.find(test_dir, withdirs=True)

['/ospool/uc-shared/public/OSG-Staff/validation',
 '/ospool/uc-shared/public/OSG-Staff/validation/README.txt',
 '/ospool/uc-shared/public/OSG-Staff/validation/test.txt',
 '/ospool/uc-shared/public/OSG-Staff/validation/test2.txt']

In [50]:
file_system.find(test_dir, detail=True)

{'/ospool/uc-shared/public/OSG-Staff/validation/README.txt': {'name': '/ospool/uc-shared/public/OSG-Staff/validation/README.txt',
  'size': None,
  'type': 'file'},
 '/ospool/uc-shared/public/OSG-Staff/validation/test.txt': {'name': '/ospool/uc-shared/public/OSG-Staff/validation/test.txt',
  'size': None,
  'type': 'file'},
 '/ospool/uc-shared/public/OSG-Staff/validation/test2.txt': {'name': '/ospool/uc-shared/public/OSG-Staff/validation/test2.txt',
  'size': None,
  'type': 'file'}}

In [53]:
with file_system.open(f"{test_dir}/README.txt", "r") as f:
    readme_contents = f.read()
print(readme_contents)

This directory is used for testing OSDF



### Cache related?

In [62]:
await file_system.get_working_cache(f"{test_dir}/test.txt")

'https://osdf-uw-cache.svc.osg-htc.org:8443/ospool/uc-shared/public/OSG-Staff/validation/test.txt'

In [64]:
print(file_system.preferred_caches)

None


### Pelican Map

[example notebook](examples/pelicanfs_example.ipynb)

In [70]:
from pelicanfs.core import PelicanFileSystem, PelicanMap
import xarray as xr


In [ ]:
fs = PelicanFileSystem("pelican://osg-htc.org")
zarr_file = PelicanMap("/chtc/PUBLIC/eturetsky/ncar-subset/ncar/monthly/cesm2LE-historical-cmip6-FLUT.zarr", fs)


In [69]:
await zarr_file

TypeError: object FSMap can't be used in 'await' expression

In [71]:
test = xr.open_dataset(zarr_file, engine="zarr")
test

TypeError: Unsupported type for store_like: 'FSMap'

In [72]:
test = xr.open_dataset(
    "osdf:///chtc/PUBLIC/eturetsky/ncar-subset/ncar/monthly/cesm2LE-historical-cmip6-FLUT.zarr", engine="zarr"
)
test

No working cache found
No working cache found
No working cache found
No working cache found


NoAvailableSource: 